In [1]:
import sys

In [9]:
sys.path.insert(0, '/home/joe/Documents/LambdaSchool/labs_ir/repos/Data-Science/data')

In [10]:
sys.path

['/home/joe/Documents/LambdaSchool/labs_ir/repos/Data-Science/data',
 '/home/joe/PythonRoot/anaconda3/envs/riskylobster/lib/python36.zip',
 '/home/joe/PythonRoot/anaconda3/envs/riskylobster/lib/python3.6',
 '/home/joe/PythonRoot/anaconda3/envs/riskylobster/lib/python3.6/lib-dynload',
 '/home/joe/PythonRoot/anaconda3/envs/riskylobster/lib/python3.6/site-packages',
 '/home/joe/PythonRoot/anaconda3/envs/riskylobster/lib/python3.6/site-packages/IPython/extensions',
 '/home/joe/.ipython']

In [4]:
sys.path.remove('')

In [14]:
import csv

In [11]:
from fin_data import DailyTimeSeries
from fracdiff import FractionalDifferentiation as fd

import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import ModelCheckpoint 

    
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

Using TensorFlow backend.


In [ ]:
def generate_df_fracdiff_target(symbol, return_type='generator', shift=1):
    # Ininialize class
    dts = DailyTimeSeries(symbol)
    df = dts.initiate()
    funs = ['totalrevenue',  
            'totalcostofrevenue', 
            'totalgrossprofit', 
            'totalpretaxincome', 
            'weightedavebasicdilutedsharesos', 
            'cashdividendspershare']
    techs = ['SMA', 'WMA', 'STOCH', 'ROC', 'AROON']
    macros = ['housing_index', 'confidence_index', 'trade_index', 'longterm_rates', 'shortterm_rates']
    df = dts.add_fundamentals(df, funs)
    df = dts.add_technicals(techs, df)
    df = dts.add_macro(df, macros)
    todrop = []
    for col in df.columns:
        if df[col].isnull().sum()/len(df) >  .25:
            print('{} is missing '.format(col), df[col].isnull().sum())
            print('{} dropped'.format(col))
            todrop.append(col)

    df = df.drop(labels=todrop, axis=1)
    df = df.dropna(axis=0)
    
    if str(symbol + '_adjusted_close') in df.columns:
        df['target'] = fd.frac_diff_ffd(df[[str(symbol + '_adjusted_close')]], .35).shift(-1)[str(symbol + '_adjusted_close')]
    else:
        df['target'] = fd.frac_diff_ffd(df[[str(symbol + '_close')]], .35).shift(-1)[str(symbol + '_close')]

    df = df.dropna()
    X = df.drop(columns='target')
    y = df[['target']].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=.8, 
                                                    shuffle=False)

    scaler = MinMaxScaler()
    scaler.fit(X_train)


    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    if return_type == 'generator':
        train_data_generator = TimeseriesGenerator(X_train, y_train, 
                                           length=30,
                                           sampling_rate=1, 
                                           stride=1, 
                                           batch_size=30)

        test_data_generator = TimeseriesGenerator(X_test, y_test, 
                                                  length=30, 
                                                  sampling_rate=1,
                                                  stride=1,
                                                  batch_size=30)
        
        return(train_data_generator, test_data_generator)
    
    elif return_type == 'Xy_trte':
        return(X_train, X_test, y_train, y_test)
    
    elif return_type == 'df':
        return(df)
    else:
        return(df)


In [21]:
csv500 = open('constituents_csv.csv','r')
sp500 = []

next(csv500)

for sym in csv.reader(csv500, delimiter=','):
    sp500.append(sym[0])

In [24]:
from decouple import config
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.timeseries import TimeSeries

In [25]:
alpha_vantage_key = config('ALPHA_VANTAGE')
ts = TimeSeries(key=alpha_vantage_key, output_format='pandas')

In [26]:
dir(ts)

['_ALPHA_VANTAGE_API_URL',
 '_ALPHA_VANTAGE_DIGITAL_CURRENCY_LIST',
 '_ALPHA_VANTAGE_MATH_MAP',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_append_type',
 '_call_api_on_func',
 '_handle_api_call',
 '_output_format',
 'get_batch_stock_quotes',
 'get_daily',
 'get_daily_adjusted',
 'get_intraday',
 'get_monthly',
 'get_monthly_adjusted',
 'get_quote_endpoint',
 'get_symbol_search',
 'get_weekly',
 'get_weekly_adjusted',
 'indexing_type',
 'key',
 'map_to_matype',
 'output_format',
 'proxy',
 'set_proxy',
 'treat_info_as_error']

In [35]:
aapl_daily = ts.get_daily_adjusted(symbol='AAPL')
aapl_daily[0]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2019-09-18,221.0100,222.8500,219.4400,222.77,222.7700,15241784.0,0.00,1.0
2019-09-17,219.9600,220.8200,219.1300,220.70,220.7000,17423847.0,0.00,1.0
2019-09-16,217.7300,220.1300,217.5600,219.90,219.9000,20750000.0,0.00,1.0
2019-09-13,220.0000,220.7900,217.0200,218.75,218.7500,39763300.0,0.00,1.0
2019-09-12,224.8000,226.4200,222.8600,223.09,223.0900,32226700.0,0.00,1.0
2019-09-11,218.0700,223.7100,217.7300,223.59,223.5900,44289600.0,0.00,1.0
2019-09-10,213.8600,216.7800,211.7100,216.70,216.7000,31777900.0,0.00,1.0
2019-09-09,214.8400,216.4400,211.0700,214.17,214.1700,27309400.0,0.00,1.0
2019-09-06,214.0500,214.4200,212.5100,213.26,213.2600,19362300.0,0.00,1.0


In [27]:
help(ts)

Help on TimeSeries in module alpha_vantage.timeseries object:

class TimeSeries(alpha_vantage.alphavantage.AlphaVantage)
 |  This class implements all the api calls to times series
 |  
 |  Method resolution order:
 |      TimeSeries
 |      alpha_vantage.alphavantage.AlphaVantage
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  get_batch_stock_quotes(self, symbols)
 |      Return multiple stock quotes with a single request.
 |      It raises ValueError when problems arise
 |      
 |      Keyword Arguments:
 |          symbols:  A tuple or list Sof symbols to query
 |  
 |  get_daily(self, symbol, outputsize='compact')
 |      Return daily time series in two json objects as data and
 |      meta_data. It raises ValueError when problems arise
 |      
 |      Keyword Arguments:
 |          symbol:  the symbol for the equity we want to get its data
 |          outputsize:  The size of the call, supported values are
 |              'compact' and 'full; the first returns th

In [36]:
ti = TechIndicators(key=alpha_vantage_key, output_format='pandas')

In [68]:
dir(ti)

['_ALPHA_VANTAGE_API_URL',
 '_ALPHA_VANTAGE_DIGITAL_CURRENCY_LIST',
 '_ALPHA_VANTAGE_MATH_MAP',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_append_type',
 '_call_api_on_func',
 '_handle_api_call',
 '_output_format',
 'get_ad',
 'get_adosc',
 'get_adx',
 'get_adxr',
 'get_apo',
 'get_aroon',
 'get_aroonosc',
 'get_atr',
 'get_bbands',
 'get_bop',
 'get_cci',
 'get_cmo',
 'get_dema',
 'get_dx',
 'get_ema',
 'get_ht_dcperiod',
 'get_ht_dcphase',
 'get_ht_phasor',
 'get_ht_sine',
 'get_ht_trendline',
 'get_ht_trendmode',
 'get_kama',
 'get_macd',
 'get_macdext',
 'get_mama',
 'get_mfi',
 'get_midpoint',
 'get_midprice',
 'get_minus_di',
 'get_minus_dm',
 'get_mom',
 

In [40]:
technicals = np.array(['get_ad',
                         'get_adosc',
                         'get_adx',
                         'get_adxr',
                         'get_apo',
                         'get_aroon',
                         'get_aroonosc',
                         'get_atr',
                         'get_bbands',
                         'get_bop',
                         'get_cci',
                         'get_cmo',
                         'get_dema',
                         'get_dx',
                         'get_ema',
                         'get_ht_dcperiod',
                         'get_ht_dcphase',
                         'get_ht_phasor',
                         'get_ht_sine',
                         'get_ht_trendline',
                         'get_ht_trendmode',
                         'get_kama',
                         'get_macd',
                         'get_macdext',
                         'get_mama',
                         'get_mfi',
                         'get_midpoint',
                         'get_midprice',
                         'get_minus_di',
                         'get_minus_dm',
                         'get_mom',
                         'get_natr',
                         'get_obv',
                         'get_plus_di',
                         'get_plus_dm',
                         'get_ppo',
                         'get_roc',
                         'get_rocr',
                         'get_rsi',
                         'get_sar',
                         'get_sma',
                         'get_stoch',
                         'get_stochf',
                         'get_stochrsi',
                         'get_t3',
                         'get_tema',
                         'get_trange',
                         'get_trima',
                         'get_trix',
                         'get_ultosc',
                         'get_willr',
                         'get_wma',])

np.save('technicals_list', technicals)

In [59]:
aapl_ti = ti.get_ad('AAPL')

In [67]:
aapl_ti[0].isnull().sum().values[0]

0

In [57]:
for tech in technicals:
    got_techs = getattr(ti, tech)(symbol='AAPL')
    print(got_techs[1]['2: Indicator'])
    print(got_techs[0].isnull().sum())

Chaikin A/D Line
Chaikin A/D Oscillator (ADOSC)
Average Directional Movement Index (ADX)
Average Directional Movement Index Rating (ADXR)
Absolute Price Oscillator (APO)
Aroon (AROON)
Aroon Oscillator (AROONOSC)
Average True Range (ATR)
Bollinger Bands (BBANDS)
Balance Of Power (BOP)
Commodity Channel Index (CCI)
Chande Momentum Oscillator (CMO)
Double Exponential Moving Average (DEMA)
Directional Movement Index (DX)
Exponential Moving Average (EMA)
Hilbert Transform - Dominant Cycle Period (HT_DCPERIOD)
Hilbert Transform - Dominant Cycle Phase (HT_DCPHASE)
Hilbert Transform - Phasor Components (HT_PHASOR)
Hilbert Transform - SineWave (HT_SINE)
Hilbert Transform - Instantaneous Trendline (HT_TRENDLINE)
Hilbert Transform - Trend vs Cycle Mode (HT_TRENDMODE)
Kaufman Adaptive Moving Average (KAMA)
Moving Average Convergence/Divergence (MACD)
MACD with Controllable MA Type (MACDEXT)
MESA Adaptive Moving Average (MAMA)
Money Flow Index (MFI)
MidPoint over period (MIDPOINT)
Midpoint Price ov

In [69]:
fe = ForeignExchange(key=alpha_vantage_key, output_format='pandas')

In [70]:
dir(fe)

['_ALPHA_VANTAGE_API_URL',
 '_ALPHA_VANTAGE_DIGITAL_CURRENCY_LIST',
 '_ALPHA_VANTAGE_MATH_MAP',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_append_type',
 '_call_api_on_func',
 '_handle_api_call',
 '_output_format',
 'get_currency_exchange_daily',
 'get_currency_exchange_intraday',
 'get_currency_exchange_monthly',
 'get_currency_exchange_rate',
 'get_currency_exchange_weekly',
 'indexing_type',
 'key',
 'map_to_matype',
 'output_format',
 'proxy',
 'set_proxy',
 'treat_info_as_error']

In [78]:
fe.get_currency_exchange_daily(from_symbol='USD', to_symbol='CNY', outputsize='full')

(            1. open  2. high  3. low  4. close
 date                                          
 2019-09-18   7.0894   7.0931  7.0821    7.0860
 2019-09-17   7.0752   7.0999  7.0711    7.0922
 2019-09-16   7.0672   7.0772  7.0622    7.0673
 2019-09-13   7.0792   7.0792  7.0787    7.0787
 2019-09-12   7.0992   7.1042  7.0646    7.0793
 ...             ...      ...     ...       ...
 2005-11-15   8.0831   8.0846  8.0823    8.0823
 2005-11-14   8.0847   8.0856  8.0800    8.0800
 2005-11-11   8.0857   8.0860  8.0847    8.0847
 2005-11-10   8.0857   8.0867  8.0845    8.0846
 2005-11-09   8.0867   8.0867  8.0867    8.0867
 
 [3530 rows x 4 columns],
 {'1. Information': 'Forex Daily Prices (open, high, low, close)',
  '2. From Symbol': 'USD',
  '3. To Symbol': 'CNY',
  '4. Output Size': 'Full size',
  '5. Last Refreshed': '2019-09-18 23:30:00',
  '6. Time Zone': 'GMT+8'})